### IMPORTS

In [1]:
import ipynb.fs.full.utils as utils
import ipynb.fs.full.blockchain as blockchain

### MIDDLEWARE STARTS

In [2]:
print('BOOTING UP...')

BOOTING UP...


### LOAD DATA FROM EXTERNAL FILES

In [3]:
settings = utils.load_json('config/settings.json')

In [4]:
latest = utils.load_json('config/latest.json')

In [5]:
print('READ EXTERNAL FILES!')

READ EXTERNAL FILES!


### CONNECT TO BLOCKCHAIN VIA WEBSOCKET

In [6]:
web3 = blockchain.connect(settings)

CONNECTED WITH GATEWAY!


### SERIALIZE MANAGER CONTRACTS

In [7]:
user_manager = blockchain.contract(latest['usermanager'], web3, settings)

In [8]:
oracle_manager = blockchain.contract(latest['oraclemanager'], web3, settings)

In [9]:
task_manager = blockchain.contract(latest['taskmanager'], web3, settings)

In [10]:
token_manager = blockchain.contract(latest['tokenmanager'], web3, settings)

In [11]:
print('MANAGER CONTRACTS SERIALIZED!')

MANAGER CONTRACTS SERIALIZED!


### INIT MANAGER CONTRACTS

In [12]:
def init_managers():
    
    # TOKEN PARAMS
    token_symbol = 'ArcaCoin'
    token_price = 5000
    token_capacity = 10000
    
    # TASK PARAMS
    task_fee = 2
    
    # TOKEN MANAGER
    token_manager.write({
        'func': 'init',
        'params': [
            token_symbol,
            token_price,
            token_capacity,
            task_manager.address
        ]
    })
    
    print('TOKEN MANAGER INITIALIZED')
    
    # TASK MANAGER
    task_manager.write({
        'func': 'init',
        'params': [
            task_fee,
            user_manager.address,
            oracle_manager.address,
            token_manager.address
        ]
    })
    
    print('TASK MANAGER INITIALIZED')
    
    # USER MANAGER
    user_manager.write({
        'func': 'init',
        'params': [task_manager.address]
    })
    
    print('USER MANAGER INITIALIZED')
    
    # DEVICE MANAGER
    oracle_manager.write({
        'func': 'init',
        'params': [
            user_manager.address,
            task_manager.address
        ]
    })
    
    print('ORACLE MANAGER INITIALIZED')
    
    return True

### INITIALIZE MANAGERS IF NECESSARY

In [13]:
def check():
    
    # CHECK STATUS
    initialized = task_manager.read('initialized')
    
    # INIT HAS BEEN PERFORMED
    if (initialized):
        print('MANAGER CONTRACTS HAVE ALREADY BEEN INITIALIZED!')

    # INIT NOW
    else:
        result = init_managers()

        if (result):
            print('\n INITIALIZATION COMPLETED!')

In [14]:
check()

TOKEN MANAGER INITIALIZED
TASK MANAGER INITIALIZED
USER MANAGER INITIALIZED
ORACLE MANAGER INITIALIZED

 INITIALIZATION COMPLETED!


### TEST -- REGISTER USER

In [17]:
user_manager.write({
    'func': 'create',
    'params': []
})

web3.exceptions.SolidityError('execution reverted: wallet is already registered')

### TEST -- REGISTER ORACLE

In [18]:
oracle_manager.write({
    'func': 'create',
    'params': [
        '617bfd20d11c3676f57be55470e6457c3acd43224b8b735d73abfbcf32b9569b',
        5
    ]
})

web3.exceptions.SolidityError('execution reverted: identifier already exists')